# EEG DataLoader

The dimensions of the training set are as follows: 4,500 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_train.

The dimensions of the testing set are as follows: 750 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_test.

You can download it from this Google Drive link: [https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing].
Please download the data and place it in your data_path at "./data."

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import csv
from torch.utils.data import DataLoader, TensorDataset

In [2]:
data_path = 'data/'

In [3]:
train_data = np.load(data_path + 'train_data.npy')
test_data = np.load(data_path + 'test_data.npy')
train_label = np.load(data_path + 'train_label.npy')
test_label = np.load(data_path + 'test_label.npy')

#To convert the data into PyTorch tensors
x_train_tensor = torch.Tensor(train_data)
y_train_tensor = torch.LongTensor(train_label)
x_test_tensor = torch.Tensor(test_data)
y_test_tensor = torch.LongTensor(test_label)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Setting GPU on your computer

In [5]:
train_dataset = TensorDataset(x_train_tensor.to(device), y_train_tensor.to(device)) # input data to Tensor dataloader
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True) #  Batch size refers to the number of data sample
test_dataset = TensorDataset(x_test_tensor.to(device), y_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=64,  drop_last=True,shuffle=False)

# Show Data

# Build simple Deep learning model

In [6]:
class EEGAutoencoderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EEGAutoencoderClassifier, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 795, 512), # Input dimention is 64 channel * 795 time point, and use 256 units for first NN layer
            nn.ReLU(), # Use ReLu function for NN training
            nn.Linear(512, 256), # 256 NN units to 128 units
            nn.ReLU(),
            nn.Linear(256, 128),#  128 NN units to 64 units
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(32, num_classes), # num_classes is 5 (hello,” “help me,” “stop,” “thank you,” and “yes”)
            nn.LogSoftmax(dim=1)  # Use LogSoftmax for multi-class classification
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.encoder(x)

        # import pdb;pdb.set_trace()
        x = self.classifier(x)
        return x

In [7]:
num_classes = 5 # setting final output class
model = EEGAutoencoderClassifier(num_classes).to(device)
criterion = nn.NLLLoss() # Use NLLLoss function to optimize

In [8]:
optimizer = optim.Adam(model.parameters(), lr=0.0001) # Setting parameters learning rate = 0.0001

In [9]:
num_epochs = 20 # setting training epochs (Number of training iterations)
for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/20, Loss: 1.5879477262496948
Epoch 2/20, Loss: 1.5736573934555054
Epoch 3/20, Loss: 1.4652020931243896
Epoch 4/20, Loss: 1.378119945526123
Epoch 5/20, Loss: 1.2584354877471924
Epoch 6/20, Loss: 1.244750738143921
Epoch 7/20, Loss: 1.1368478536605835
Epoch 8/20, Loss: 0.8492183685302734
Epoch 9/20, Loss: 1.0124014616012573
Epoch 10/20, Loss: 0.5366473197937012
Epoch 11/20, Loss: 0.5674989819526672
Epoch 12/20, Loss: 0.33723941445350647
Epoch 13/20, Loss: 0.3028312623500824
Epoch 14/20, Loss: 0.23253951966762543
Epoch 15/20, Loss: 0.1574769765138626
Epoch 16/20, Loss: 0.13561952114105225
Epoch 17/20, Loss: 0.09890972822904587
Epoch 18/20, Loss: 0.10339082777500153
Epoch 19/20, Loss: 0.05302770063281059
Epoch 20/20, Loss: 0.0655115619301796


In [10]:
model.eval() # Evaluate your model
correct = 0
total = 0
inputs = []
outcomes = []
with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        outcomes+=predicted
        inputs += labels

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')
#print(outcomes)

Test Accuracy: 50.99%


In [11]:
def output_interpret(out):
    i = out.item()
    if i == 0:
        return "hello"
    if i == 1:
        return "help me"
    if i == 2:
        return "stop"
    if i == 3:
        return "thank you"
    if i == 4:
        return "yes"
                
    return "fail"

In [12]:
def data_and_predictions():
    pred = list(map(output_interpret,outcomes))
    data = list(map(output_interpret,inputs))
    result = []
    
    for i in range(0,len(pred)):
        result = result + [(data[i] , pred[i])]
    
    return result

print(data_and_predictions())

[('thank you', 'yes'), ('yes', 'thank you'), ('help me', 'help me'), ('stop', 'yes'), ('help me', 'yes'), ('yes', 'stop'), ('thank you', 'thank you'), ('help me', 'thank you'), ('stop', 'stop'), ('stop', 'hello'), ('stop', 'stop'), ('hello', 'hello'), ('stop', 'yes'), ('help me', 'help me'), ('help me', 'help me'), ('stop', 'stop'), ('help me', 'help me'), ('yes', 'yes'), ('yes', 'yes'), ('help me', 'yes'), ('hello', 'yes'), ('help me', 'help me'), ('hello', 'help me'), ('stop', 'yes'), ('thank you', 'stop'), ('hello', 'hello'), ('thank you', 'thank you'), ('thank you', 'yes'), ('thank you', 'stop'), ('thank you', 'yes'), ('hello', 'hello'), ('thank you', 'thank you'), ('yes', 'yes'), ('stop', 'stop'), ('yes', 'yes'), ('hello', 'yes'), ('help me', 'stop'), ('yes', 'yes'), ('yes', 'yes'), ('hello', 'stop'), ('yes', 'yes'), ('help me', 'hello'), ('hello', 'yes'), ('stop', 'stop'), ('yes', 'yes'), ('thank you', 'stop'), ('hello', 'yes'), ('hello', 'yes'), ('stop', 'stop'), ('thank you', '

In [13]:
data = data_and_predictions()
with open('model_outcomes.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['data','prediction'])
    for row in data:
        csv_out.writerow(row)